In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model


2023-10-05 21:27:06.035748: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 21:27:06.924318: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 21:27:06.924365: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 21:27:06.930320: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-05 21:27:07.463782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 21:27:07.467153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
from tensorflow.keras.datasets import mnist
from pyspark.sql.types import StructType,StructField, StringType, FloatType, ArrayType

In [3]:
import pandas as pd

In [4]:
# train, test load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [5]:
# PY SPARK VERSION OF THE DATAFRAME
# def get_ps_dataFrame(x_data,y_data):
#     data_unified = [( x_data[i].tolist(), str(y_data[i])) for i in range(len(x_data))]
#     schema = StructType([ 
#         StructField("image",ArrayType(ArrayType(FloatType())),False), 
#         StructField("class", StringType(), False) 
#       ])
#     df = spark.createDataFrame(data=data_unified,schema=schema)
#     return df
# train_df = get_ps_dataFrame(x_train, y_train)

In [6]:
# print((df.count(), len(df.columns)))

In [7]:
def get_pd_dataFrame(x_data,y_data):
    return pd.DataFrame(data={'image':x_data.tolist(), 'class':y_data.tolist()})
train_df = get_pd_dataFrame(x_train, y_train)

In [8]:
def augment_data_cross_same_class_pair(df:pd.DataFrame):
    sorted_df = df.sort_values("class").reset_index(drop=True)
    shuffled_sorted_df = sorted_df.sample(frac=1).reset_index(drop=True).sort_values("class").reset_index(drop=True)
    shuffled_sorted_df.rename(columns={'image':'imagePair',"class": "classPair"}, inplace=True)
    result = pd.concat([sorted_df, shuffled_sorted_df], axis=1)
    return result[['image','imagePair', 'class']].sample(frac=1).reset_index(drop=True)

same_class_cross_df = augment_data_cross_same_class_pair(train_df)

In [9]:
import tensorflow as tf
def series_to_tensor(series):
    tensor_list = series.apply(lambda x: tf.constant(x))
    return tf.stack(tensor_list)
    
def prepare_train_traditional_autoencoder_data(train_df):
    image_tensor = series_to_tensor(train_df['image'])
    in_x_train = image_tensor
    out_x_train = image_tensor
    return [in_x_train, out_x_train]

def prepare_train_cross_same_class_autoencoder_data(train_df, additional_same_class_df):
    original_df =  train_df.copy()
    original_df["imagePair"] = original_df["image"]
    concat_df = pd.concat([original_df, additional_same_class_df])
    concat_df = concat_df.sample(frac=1).reset_index(drop=True)
    in_x_train = series_to_tensor(concat_df['image'])
    out_x_train = series_to_tensor(concat_df['imagePair'])
    return [in_x_train, out_x_train]

In [10]:
[trad_ae_x_in,  trad_ae_x_out]= prepare_train_traditional_autoencoder_data(train_df)

[cross_same_class_ae_x_in,  cross_same_class_ae_x_out]= prepare_train_cross_same_class_autoencoder_data(train_df, same_class_cross_df)



In [61]:
from keras import layers
class Autoencoder(keras.Model):
    def __init__(self, latent_dim, input_shape ):
        super(Autoencoder, self).__init__(input_shape,input_shape)
        self.latent_dim = latent_dim
        self.shape = input_shape
        self.encoder = tf.keras.Sequential([
            input_shape,
            layers.Conv2D(16, (3, 3), activation='relu'),
            layers.MaxPooling2D((2,2), padding='same'),
            layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2,2), padding='same'),
            layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D((2,2), padding='same'),
        ])
        encoder_output_shape = self.encoder.layers[-1].output_shape
        self.decoder = tf.keras.Sequential([
            layers.Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=encoder_output_shape[1:]),
            layers.UpSampling2D((2, 2)),
            layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
            layers.UpSampling2D((2, 2)),
            layers.Conv2D(16, (3, 3), activation='relu'),
            layers.UpSampling2D((2, 2)),
            layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'),
        ])

    def call(self, inputs,training = False):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded


input_shape = keras.Input(shape=(28, 28, 1))
latent_dim = 2
ae = Autoencoder(latent_dim, input_shape)
ae.compile(optimizer='adam', loss='mse')

In [62]:
ae.summary()

Model: "autoencoder_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 sequential_10 (Sequential)  (None, 4, 4, 8)           1904      
                                                                 
 sequential_11 (Sequential)  (None, 28, 28, 1)         2481      
                                                                 
Total params: 4385 (17.13 KB)
Trainable params: 4385 (17.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
# train autoencoder
from keras.callbacks import ModelCheckpoint

# specify the path where you want to save your models
checkpoint_file_path =  './checkpoints/ae'

# create a model checkpoint callback
checkpoint = ModelCheckpoint(
    checkpoint_file_path, 
    monitor='val_loss',
    verbose=0, 
    save_weights_only=True, 
    save_best_only=False,
    mode='min', 
    save_freq="epoch")

history = ae.fit(trad_ae_x_in, trad_ae_x_in,
                epochs=3,#100
                batch_size=1000,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[checkpoint] ,
                workers=8,
                use_multiprocessing=True)

Epoch 1/3
60/60 [==============================] - 14s 219ms/step - loss: 0.1354 - val_loss: 0.1140
Epoch 2/3
60/60 [==============================] - 13s 211ms/step - loss: 0.1120 - val_loss: 0.1140
Epoch 3/3
60/60 [==============================] - 13s 216ms/step - loss: 0.1120 - val_loss: 0.1140


In [63]:

ae.load_weights(checkpoint_file_path)

<bound method Model.predict of <keras.src.engine.sequential.Sequential object at 0x7fd166917730>>

In [67]:
# plot loss ae
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

AttributeError: 'NoneType' object has no attribute 'history'

In [18]:
# prepare data aumentated for same class match
# train VAE 
# plot loss vae

In [97]:
# calculate encoded_vector_ae using autoencoder
# calculate encoded_vector_vae using Variational autoencoder

In [99]:
# create function to calculate metric topK class metric
# calculateMetric(encoded_vector_ae)
# calculateMetric(encoded_vector_vae)

In [100]:
# data visualisation metrics

In [101]:
# discuss results

In [ ]:
# if there is time, apply in another dataset